#Assignment 2
1. This assignment is due in two weeks, at 23:59 Feb 11th 2022.
2. There are two files to submit. Please name your .py and .ipynb file using your student number as Axxxxxx.py, Axxxxxxx.ipynb and submit it to Luminus->assignments->submissions->assignment2

# Part 1: Policy Gradients

You will implement the vanilla policy gradients algorithm, also referred to as
REINFORCE.

## Review

In policy gradients, the objective is to learn a parameter $\theta^*$ that
maximizes the following objective:

\begin{equation}
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta(\tau)}[R(\tau)]
\end{equation}

where $\tau = (s_1,a_1,s_2,\ldots,s_{T-1},a_{T-1},s_T)$ is a *trajectory*
(also referred to as an *episode*), and factorizes as

\begin{equation}
\pi_\theta(\tau) = p(s_1)\pi_\theta(a_1|s_1)\prod_{t=2}^{T} p(s_t|s_{t-1},a_{t-1})\pi_\theta(a_t|s_t)
\end{equation}

and $R(\tau)$ denotes the full trajectory reward $R(\tau) = \sum_{t=1}^{T}
r(s_t,a_t)$ with $r(s_t,a_t)$ the rewards at the individual time steps.

In policy gradients, we directly apply the gradient $\nabla_\theta$ to
$J(\theta)$. In order to do so, we require samples of trajectories, meaning that
we now denote them as $\tau_i$ for the $i$th trajectory, and have $\tau_i =
(s_{i1},a_{i1},s_{i2},\ldots,s_{iT})$. When we approximate the gradient with
samples, we get:

\begin{align*}
\nabla_\theta J(\theta) &\approx \frac{1}{N} \sum_{i=1}^N \nabla_\theta \log \pi_\theta(\tau_i) R(\tau_i) \\
&= \frac{1}{N}\sum_{i=1}^N \left( \sum_{t=1}^T \nabla_\theta \log \pi_\theta(a_{it}|s_{it}) \right)  \left( \sum_{t=1}^{T} r(s_{it},a_{it}) \right)
\end{align*}

Multiplying a discount factor $\gamma$ to the rewards can be interpreted as
encouraging the agent to focus on rewards closer in the future, which can also
be thought of as a means for reducing variance (because there are more
possible futures further into the future). The discount factor can be
incorporated in two ways, from the full trajectory:

\begin{equation}
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N
\left( \sum_{t=1}^T \nabla_\theta \log \pi_\theta(a_{it}|s_{it}) \right) 
\left( \sum_{t=1}^T \gamma^{t-1} r(s_{it},a_{it}) \right)
\end{equation}

and from the reward to go:

\begin{equation}
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N
\left( \sum_{t=1}^T \nabla_\theta \log \pi_\theta(a_{it}|s_{it}) \right) 
\left( \sum_{t'=t}^T \gamma^{t'-t} r(s_{it},a_{it}) \right)
\end{equation}

**In this assignment, we only focus on the first version: full tragectory.**



# Policy Gradients Implementation


**You will need to write code in `PolicyGradient.ipynb`. The places where you need to write code are
 clearly indicated with the comments `START OF YOUR CODE` and
`END OF YOUR CODE`. 
You do not need to change any other files for this part of the assignment.**

The dataflow of the code is structured like this: 

- Set Up Hyperparameters and environment.
- Build a MLP model for policy learning.
- Initialize the agent, such as define the policy network and optimizer.
- Forward Computation: Sample trajectories by conducting an action given an observation from the environment, and calculate sum of rewards in each trajectory, That includes `sample_action`, `sample_trajectory`, `sample_trajectories` and `sum_of_rewards`.
- Backward Computation: Optimize the policy network based on the update rule. That contains `compute_advantage`, `estimate_return`, `get_log_prob` , `update_parameters`.

## Problem 1: data sampling

You need to implement any parts with a "Problem 1" header in the code. Here's what you need to do:

- 1. Implement `sample_action`, which samples an action from $\pi_\theta(a|s)$. This operation will be called in `sample_trajectories`.
- 2. Implement `sample_trajectory`, you need to call `sample_action` to obtain current action.
- 3. Implement `sum_of_rewards`, which is the Monte Carlo estimation of the Q function. You need to estimate the q-value of each path and return a single vector for the estimated q values whose length is the sum of the lengths of the paths.

## Problem 2: apply policy gradient
You only need to implement the parts with the "Problem 2" header.

- **Estimate return**: in `estimate_return`, normalize the advantages to have a mean of zero and a standard deviation of one.  This is a trick for reducing variance.
- Implement `get_log_prob` to obtain $\log \pi_\theta(a_{it}|s_{it})$: Given an action that the agent took in the environment, this computes the log probability of that action under $\pi_\theta(a|s)$. This will be used in the parameters update: 

\begin{equation}
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N
\left( \sum_{t=1}^T \nabla_\theta \log \pi_\theta(a_{it}|s_{it}) \right) 
\left( \sum_{t=1}^T \gamma^{t-1} r(s_{it},a_{it}) \right)
\end{equation}

- **Update parameters**: In `update_parameters`, using the update operation `optimizer.step()` to update the parameters of the policy. You firstly need to create loss value with the inputs.



# Environment Introduction: 


##[CartPole-v0](https://gym.openai.com/envs/CartPole-v0/): 
This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson in ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077). A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum starts upright, and the goal is to prevent it from falling over by increasing and reducing the cart's velocity.

### Observation Space
The observation is a `ndarray` with shape `(4,)` where the elements correspond to the following:

| Num | Observation           | Min                  | Max                |
|-----|-----------------------|----------------------|--------------------|
| 0   | Cart Position         | -4.8*                |  4.8*                |
| 1   | Cart Velocity         | -Inf                 | Inf                |
| 2   | Pole Angle            | ~ -0.418 rad (-24°)** | ~ 0.418 rad (24°)** |
| 3   | Pole Angular Velocity | -Inf                 | Inf                 |

- `*`: the cart x-position can be observed between `(-4.8, 4.8)`, but an episode terminates if the cart leaves the
    `(-2.4, 2.4)` range.
- `**`: Similarly, the pole angle can be observed between  `(-.418, .418)` radians or precisely **±24°**, but an episode is
    terminated if the pole angle is outside the `(-.2095, .2095)` range or precisely **±12°**

### Action Space
The agent take a 1-element vector for actions.
The action space is `(action)` in `[0, 1]`, where `action` is used to push
the cart with a fixed amount of force:

| Num | Action                 |
|-----|------------------------|
| 0   | Push cart to the left  |
| 1   | Push cart to the right |

Note: The amount the velocity is reduced or increased is not fixed as it depends on the angle the pole is pointing.
This is because the center of gravity of the pole increases the amount of energy needed to move the cart underneath it

### Rewards
Reward is 1 for every step taken, including the termination step.
### Starting State
All observations are assigned a uniform random value between (-0.05, 0.05).
### Episode Termination
The episode terminates of one of the following occurs:
1. Pole Angle is more than ±12°
2. Cart Position is more than ±2.4 (center of the cart reaches the edge of the display)
3. Episode length is greater than 200. 


In [ ]:
!pip install gym==0.10.5

In [ ]:
import numpy as np
import gym
import os
import time
import inspect
import sys
from multiprocessing import Process
import torch
from torch import nn as nn
import torch.nn.functional as F

## Set Up Hyperparameters

In [ ]:
env_name = 'CartPole-v0'
# exp_name = 'vpg'
render = False
animate = render
discount = 1.0
n_iter = 101
batch_size = 1000
ep_len = -1.
learning_rate = 5e-3
reward_to_go = False
dont_normalize_advantages = False
seed = 1
n_experiments = 1
max_path_length = ep_len if ep_len > 0 else None
min_timesteps_per_batch = batch_size
gamma = discount
normalize_advantages = not(dont_normalize_advantages)

## Set Up Environment

In [ ]:
#========================================================================================#
# Set Up Env
#========================================================================================#

# Make the gym environment
env = gym.make(env_name)

# Set random seeds
torch.manual_seed(seed)
np.random.seed(seed)
env.seed(seed)

# Maximum length for episodes
max_path_length = max_path_length or env.spec.max_episode_steps

# Is this env continuous, or self.discrete? In this assignment, we only consider discrete action space.
discrete = isinstance(env.action_space, gym.spaces.Discrete)

# Observation and action sizes
ob_dim = env.observation_space.shape[0]
ac_dim = env.action_space.n if discrete else env.action_space.shape[0]

## Build a MLP model for policy learning.

In [ ]:
class MLP(nn.Module):

    def __init__(self, input_size, num_actions):
        super(MLP, self).__init__()
        self.dense1 = nn.Linear(input_size, 32)
        self.dense2 = nn.Linear(32, 32)
        self.dense3 = nn.Linear(32, num_actions)

    def forward(self, x):
        x = F.tanh(self.dense1(x))
        x = F.tanh(self.dense2(x))
        out = F.softmax(self.dense3(x))
        return out


## Initialize Agent
    

In [ ]:
policy_net = MLP(input_size=ob_dim, num_actions=ac_dim)
optimizer = torch.optim.Adam(policy_net.parameters(), lr=learning_rate)


## Data Sampling

In [ ]:
def sample_action(policy_parameters):
    """
    Stochastically sampling from the policy distribution

    arguments:
        policy_parameters: logits of a categorical distribution over actions
                sy_logits_na: (batch_size, self.ac_dim)

    returns:
        sy_sampled_ac: (batch_size,)
    """

    sy_logits_na = policy_parameters
    #========================================================================================#
    #                           ----------PROBLEM 1----------
    #========================================================================================#
    # Stochastically sampling an action from the policy distribution $\pi_\theta(a|s)$.
    # ------------------------------------------------------------------
    # START OF YOUR CODE
    # ------------------------------------------------------------------




    # ------------------------------------------------------------------
    # END OF YOUR CODE
    # ------------------------------------------------------------------

    return sy_sampled_ac

In [ ]:
def sample_trajectory(env):
    ob = env.reset()
    obs, acs, rewards = [], [], []
    steps = 0
    while True:

        obs.append(ob)
        #====================================================================================#
        #                           ----------PROBLEM 1----------
        #====================================================================================#
        # obtain the action 'ac' for current observation 'ob'
        # ------------------------------------------------------------------
        # START OF YOUR CODE
        # ------------------------------------------------------------------




        # ------------------------------------------------------------------
        # END OF YOUR CODE
        # ------------------------------------------------------------------
        ac = ac.numpy()[0]
        acs.append(ac)
        ob, rew, done, _ = env.step(ac)
        rewards.append(rew)
        steps += 1
        if done or steps > max_path_length:
            break
    path = {"observation" : np.array(obs, dtype=np.float32),
            "reward" : np.array(rewards, dtype=np.float32),
            "action" : np.array(acs, dtype=np.float32)}
    return path


In [ ]:
def sample_trajectories(itr, env):
    """Collect paths until we have enough timesteps, as determined by the
    length of all paths collected in this batch.
    """
    timesteps_this_batch = 0
    paths = []
    while True:
        path = sample_trajectory(env)
        paths.append(path)
        timesteps_this_batch += len(path["reward"])
        if timesteps_this_batch > min_timesteps_per_batch:
            break
    return paths, timesteps_this_batch

For sum of rewards, we use the total discounted reward summed over entire trajectory (regardless of which time step the Q-value should be for).

In [ ]:
def sum_of_rewards(re_n):
    """ Monte Carlo estimation of the Q function.

    let sum_of_path_lengths be the sum of the lengths of the paths sampled from
        the function sample_trajectories
    let num_paths be the number of paths sampled from sample_trajectories

    arguments:
        re_n: length: num_paths. Each element in re_n is a numpy array
            containing the rewards for the particular path

    returns:
        q_n: shape: (sum_of_path_lengths). A single vector for the estimated q values
            whose length is the sum of the lengths of the paths
    ----------------------------------------------------------------------------------

    Your code should construct numpy arrays for Q-values which will be used to compute
    advantages.


    You will write code for trajectory-based PG: 

          We use the total discounted reward summed over
          entire trajectory (regardless of which time step the Q-value should be for).

          For this case, the policy gradient estimator is

              E_{tau} [sum_{t=0}^T grad log pi(a_t|s_t) * Ret(tau)]

          where

              tau=(s_0, a_0, ...) is a trajectory,
              Ret(tau) = sum_{t'=0}^T gamma^t' r_{t'}.

          Thus, you should compute

              Q_t = Ret(tau)

    Store the Q-values for all timesteps and all trajectories in a variable 'q_n',
    like the 'ob_no' and 'ac_na' above.
    """
    #====================================================================================#
    #                           ----------PROBLEM 1----------
    #====================================================================================#
    # q_n: A single vector for the estimated q values whose length is the sum of the lengths of the paths.
    # Q-values: Q_t = Ret(tau) = sum_{t'=0}^T gamma^t' r_{t'}. 
    # Store the Q-values for all timesteps and all trajectories in a variable 'q_n'.
    # ------------------------------------------------------------------
    # START OF YOUR CODE
    # ------------------------------------------------------------------



    # # ------------------------------------------------------------------
    # END OF YOUR CODE
    # ------------------------------------------------------------------
    return q_n

## Apply Policy Gradient

We firstly need to estimate return `estimate_return` and calculate log probability of actions `get_log_prob`. Then we can update parameters based on the rule:

\begin{equation}
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N
\left( \sum_{t=1}^T \nabla_\theta \log \pi_\theta(a_{it}|s_{it}) \right) 
\left( \sum_{t=1}^T \gamma^{t-1} r(s_{it},a_{it}) \right)
\end{equation}

In [ ]:
def compute_advantage(ob_no, q_n):
  
    adv_n = q_n.copy()
    return adv_n

In [ ]:
def estimate_return(ob_no, re_n):
    """ Estimates the returns over a set of trajectories.

    let sum_of_path_lengths be the sum of the lengths of the paths sampled from
        sample_trajectories
    let num_paths be the number of paths sampled from sample_trajectories

    arguments:
        ob_no: shape: (sum_of_path_lengths, ob_dim)
        re_n: length: num_paths. Each element in re_n is a numpy array
            containing the rewards for the particular path

    returns:
        q_n: shape: (sum_of_path_lengths). A single vector for the estimated q values
            whose length is the sum of the lengths of the paths
        adv_n: shape: (sum_of_path_lengths). A single vector for the estimated
            advantages whose length is the sum of the lengths of the paths
    """
    q_n = sum_of_rewards(re_n)
    adv_n = compute_advantage(ob_no, q_n)
    #====================================================================================#
    #                           ----------PROBLEM 2----------
    # Advantage Normalization
    #====================================================================================#
    if normalize_advantages:
        # On the next line, implement a trick which is known empirically to reduce variance
        # in policy gradient methods: normalize adv_n to have mean zero and std=1.
        # ------------------------------------------------------------------
        # START OF YOUR CODE
        # ------------------------------------------------------------------



        # ------------------------------------------------------------------
        # END OF YOUR CODE
        # ------------------------------------------------------------------
    return q_n, adv_n

In [ ]:
def get_log_prob(policy_parameters, sy_ac_na):
    """
    Computing the log probability of a set of actions that were actually taken according to the policy

    arguments:
        policy_parameters: logits of a categorical distribution over actions
                sy_logits_na: (batch_size, self.ac_dim)

        sy_ac_na: (batch_size,)

    returns:
        sy_logprob_n: (batch_size)

    Hint:
        For the discrete case, use the log probability under a categorical distribution.
    """

    sy_logits_na = policy_parameters
    #========================================================================================#
    #                           ----------PROBLEM 2----------
    #========================================================================================#
    # sy_logprob_n = \sum_{t=1}^T \log \pi_\theta(a_{it}|s_{it})
    # ------------------------------------------------------------------
    # START OF YOUR CODE
    # ------------------------------------------------------------------



    # ------------------------------------------------------------------
    # END OF YOUR CODE
    # ------------------------------------------------------------------
    return sy_logprob_n

In [ ]:
def update_parameters(ob_no, ac_na, q_n, adv_n):
    """
    Update the parameters of the policy and (possibly) the neural network baseline,
    which is trained to approximate the value function.

    arguments:
        ob_no: shape: (sum_of_path_lengths, ob_dim)
        ac_na: shape: (sum_of_path_lengths).
        q_n: shape: (sum_of_path_lengths). A single vector for the estimated q values
            whose length is the sum of the lengths of the paths
        adv_n: shape: (sum_of_path_lengths). A single vector for the estimated
            advantages whose length is the sum of the lengths of the paths

    returns:
        nothing
    """
    #====================================================================================#
    #                           ----------PROBLEM 2----------
    #====================================================================================#
    # Performing the Policy Update based on the current batch of rollouts.
    # 
    # ------------------------------------------------------------------
    # START OF YOUR CODE
    # ------------------------------------------------------------------



    # ------------------------------------------------------------------
    # END OF YOUR CODE
    # ------------------------------------------------------------------

## Training Loop.

In [ ]:
print('Running experiment with seed %d'%seed)

start = time.time()

total_timesteps = 0

return_data = []

for itr in range(n_iter):

    paths, timesteps_this_batch = sample_trajectories(itr, env)
    total_timesteps += timesteps_this_batch

    # Build arrays for observation, action for the policy gradient update by
    # concatenating across paths
    ob_no = np.concatenate([path["observation"] for path in paths])
    ac_na = np.concatenate([path["action"] for path in paths])

    re_n = [path["reward"] for path in paths]

    q_n, adv_n = estimate_return(ob_no, re_n)


    update_parameters(ob_no, ac_na, q_n, adv_n)

    # Log diagnostics
    returns = [path["reward"].sum() for path in paths]

    if itr%10 == 0:
        print("********** Iteration %i ************"%itr)
        ep_lengths = [len(path["reward"]) for path in paths]
        print("Time: ", time.time() - start)
        print("Iteration: ", itr)
        print("AverageReturn: ", np.mean(returns))
        print("StdReturn: ", np.std(returns))
        print("MaxReturn: ", np.max(returns))
        print("MinReturn", np.min(returns))
        print("EpLenMean: ", np.mean(ep_lengths))
        print("EpLenStd: ", np.std(ep_lengths))
        print("TimestepsThisBatch: ", timesteps_this_batch)
        print("TimestepsSoFar: ", total_timesteps)
    return_data.append(np.mean(returns))

## Plot Average-Return curve.





In [ ]:
import matplotlib.pyplot as plt
plt.plot(return_data)
plt.xlabel("Iterations")
plt.ylabel("Average Return")